In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence

In [2]:
#https://suzyahyah.github.io/pytorch/2019/07/01/DataLoader-Pad-Pack-Sequence.html

In [3]:
word_list = []
target_list=  []
word_str = ""
target_str = ""
with open(r"C:\Users\Ant PC\Music\dataset\FIN5.txt",'r', encoding="utf8") as myfile:    
    for i, line in enumerate(myfile):        
        if ((len(line)>0 and i>0) and line not in ['\n', '\r\n']) :            
            if (len(line)>0):
                word_str = word_str+" "+line.split()[0]
                target_str = target_str+" "+line.split()[-1]                    
        if line in ['\n', '\r\n']:            
            word_list.append(word_str[1:])
            target_list.append(target_str[1:])
            word_str=""
            target_str=""

In [4]:
for wl,tl in zip(word_list[1:],target_list[1:]):
    assert len(wl.split()) == len(wl.split())

In [5]:
emb_dim = 100


def load_glove(path):
    file = open(path,'r',encoding='utf-8')    
    glove_emb = {} 
    word2ind ={}
    ind2word = {}
    # Include padding to the glove vocab
    glove_emb["<pad>"] = np.zeros(emb_dim,)
    emb_vec = [[0]*emb_dim]
    word2ind["<pad>"] = 0
    ind2word[0] = "<pad>"
    
    for i,text in enumerate(file):    
        i = i+1
        line = text.split()
        word = line[0]
        emb = np.array([float(val) for val in line[1:]])
        embl = [float(val) for val in line[1:]]        
        glove_emb[word] = emb     
        tmp_emb_vec = emb.reshape(1,-1)
        emb_vec.append(embl)
        word2ind[word] = i
        ind2word[i] = word
        
    #emb_vec = np.array(emb_vec)
    return glove_emb,emb_vec,word2ind,ind2word
    
    
initglove_emb_dict,initglove_emb_list,initword2ind,initind2word = load_glove(r"C:\Users\Ant PC\Music\NER-LSTM-CNN-Pytorch-master\NER-LSTM-CNN-Pytorch-master\data\glove.6B.100d.txt")


In [6]:
text = word_list[1:] # First word is blank

In [7]:
def get_emb_vect(text,glove_emb_dict,glove_emb_list,word2ind,ind2word):
    words_NOT_found = 0
    embedding_dim = 100
    for sent in text:
        for word in sent.split():        
            word = word.lower()
            try: 
                glove_emb_dict[word]            
            except KeyError:
                words_NOT_found += 1            
                glove_emb_dict[word] = np.random.normal(scale=0.6, size=(embedding_dim,)) # scale is the S.D of the Gaussian Distribution(-1,1)
                glove_emb_list.append(list(glove_emb_dict[word]))
                word2ind[word] = len(glove_emb_dict)-1
                ind2word[len(glove_emb_dict)-1] = word
    glove_emb_vec = np.array(glove_emb_list)
    return glove_emb_vec,glove_emb_dict,word2ind,ind2word

In [8]:
glove_emb_vec,glove_emb_dict,word2ind,ind2word = get_emb_vect(text,initglove_emb_dict,initglove_emb_list,initword2ind,initind2word)

In [9]:
target = target_list[1:]
tind2word ={}
tword2ind = {}
uniq_words = ""
maxlen = 0
for sent in target:    
    uniq_words = uniq_words+ " " +sent    
    if len(sent.split()) > maxlen:
        maxlen = len(sent.split())        

uniq_words = set(uniq_words.strip().split())
tind2word[0] = "<pad>"
tword2ind["<pad>"] = 0
for i,word in enumerate(uniq_words):    
    i = i+1
    tind2word[i] = word.lower()
    tword2ind[word.lower()] = i

In [10]:
input_seq = text
target_seq = target

In [19]:
len(tind2word)

6

In [11]:
"""Now we can convert our input and target sequences to sequences of integers instead of characters by
mapping them using the dictionaries we created above. This will allow us to one-hot-encode our input sequence subsequently."""

for i in range(len(text)):
    input_seq[i] = [word2ind[word.lower()] for word in input_seq[i].split()]
    target_seq[i] = [tword2ind[word.lower()] for word in target_seq[i].split()]

In [12]:
class dataset(Dataset):
    def __init__(self,trainlist,targetlist):
        self.trainlist = trainlist
        self.targetlist = targetlist
    def __len__(self):
        return len(self.trainlist)
    def __getitem__(self,idx):
        return self.trainlist[idx],self.targetlist[idx]
train_seqdataset = dataset(input_seq[:1000],target_seq[:1000])
valid_seqdataset = dataset(input_seq[1000:],target_seq[1000:])

In [13]:
len(input_seq)

1168

In [14]:
def pad_collate(batch):
  (xx, yy) = zip(*batch)    
  xx = list(xx)  
  yy = list(yy)  
  x_lens = [len(x) for x in xx]
  y_lens = [len(y) for y in yy]  
  xx_tensor = [torch.LongTensor(x) for x in xx]
  yy_tensor = [torch.LongTensor(y) for y in yy]
  xx_pad = pad_sequence(xx_tensor, batch_first=True, padding_value=0)
  yy_pad = pad_sequence(yy_tensor, batch_first=True, padding_value=0)
  xx_embed = glove_emb_vec[xx_pad]
  return xx_embed,xx_pad, yy_pad, x_lens, y_lens

train_data_loader = DataLoader(dataset=train_seqdataset, batch_size=32, shuffle=True, collate_fn=pad_collate)
valid_data_loader = DataLoader(dataset=valid_seqdataset, batch_size=32, shuffle=True, collate_fn=pad_collate)

In [26]:
for xx_embed,xx_pad, yy_pad, x_lens, y_lens in iter(train_data_loader):
    xemb = xx_embed
    xppad = xx_pad
    yppad = yy_pad

In [27]:
mask = yppad !=  0

In [29]:
yppad

tensor([[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
        [2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5

In [30]:
mask

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True, False, False, False, False, False, False, False,
         False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True

In [31]:
yppad[mask]

tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 4, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

In [32]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [33]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        #self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        #self.batch_size = batch_size

        #Defining the layers
        # RNN Layer
        self.rnn = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True,bidirectional=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim*2, output_size)
    
    def forward(self, x,x_lens):
        
        batch_size = x.size(0)
        #x_embed = self.embedding(x)
        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        #Initializing hidden state for first input using method defined below
        hidden,cin = self.init_hidden(batch_size)
        #x = x.detach()
        # Passing in the input and hidden state into the model and obtaining outputs
        #out, (hidden,cout) = self.rnn(x, (hidden,cin))        
        output_packed, (hidden,cout) = self.rnn(x_packed, (hidden,cin))
        output_padded, output_lengths = pad_packed_sequence(output_packed, batch_first=True)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = output_padded.contiguous().view(-1, self.hidden_dim*2)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers*2, batch_size, self.hidden_dim,dtype=torch.float).to(device)
        cin = torch.zeros(self.n_layers*2, batch_size, self.hidden_dim,dtype=torch.float).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden,cin

In [34]:
output_tag = len(tind2word)
vocab_size = len(ind2word)
embedding_dim = 100
# Instantiate the model with hyperparameters
model = Model(input_size=embedding_dim, output_size=output_tag, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 25
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [38]:
# Training Run
#input_seq = x_embed.to(device)
#from sklearn.metrics import f1_score

for epoch in range(1, n_epochs + 1):
    model.train()
    train_loss = 0
    for _,(xembed,x_padded, y_padded, x_lens, y_lens) in enumerate(train_data_loader):        
        #input_seq = x_padded.to(device)
        input_seq = glove_emb_vec[x_padded]
        input_seq = torch.tensor(input_seq,dtype=torch.float).to(device)
        target_seq = y_padded.to(device)
        #x_lens = torch.tensor(x_lens,dtype=torch.float).to(device)
        y_lens = y_lens    
        optimizer.zero_grad() # Clears existing gradients from previous epoch        
        output, hidden = model(input_seq,x_lens)
        output = output.to(device)    
        loss = criterion(output, target_seq.view(-1).long())
        loss.backward() # Does backpropagation and calculates gradients
        optimizer.step() # Updates the weights accordingly
        train_loss = train_loss + loss        
    train_avg_loss = train_loss/len(train_data_loader)
    if epoch%10 == 0:
        print("Train Loss : {}".format(train_avg_loss))
    model.eval()
    valid_loss = 0 
    for _,(xembed,x_padded, y_padded, x_lens, y_lens) in enumerate(valid_data_loader):        
        #input_seq = x_padded.to(device)
        input_seq = glove_emb_vec[x_padded]
        input_seq = torch.tensor(input_seq,dtype=torch.float).to(device)
        target_seq = y_padded.to(device)
        out, hidden = model(input_seq,x_lens)
        vloss = criterion(out, target_seq.view(-1).long())
        valid_loss = valid_loss+vloss
    valid_avg_loss = valid_loss/len(valid_data_loader)
    if epoch%10 == 0:
        print("Valid Loss : {}".format(valid_avg_loss))
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Train Loss : 0.019697677344083786
Valid Loss : 0.04669174551963806
Epoch: 10/25............. Loss: 0.0620
Train Loss : 0.005179248284548521
Valid Loss : 0.06322278082370758
Epoch: 20/25............. Loss: 0.0018


In [39]:
prob = nn.functional.softmax(out, dim=1).data

In [41]:
prob.shape

torch.Size([320, 6])

In [47]:
arg_word_ind = torch.argmax(prob, dim=-1)

In [48]:
arg_word_ind.shape

torch.Size([320])

In [49]:
_,word_ind = torch.max(prob, dim=-1) 

In [50]:
word_ind.shape

torch.Size([320])

In [53]:
y_padded.reshape(-1).shape

torch.Size([320])

In [44]:
y_padded.shape

torch.Size([8, 40])

In [19]:
def predict(model, text,do_print=False):
    or_text = text.copy()
    glove_emb_vec,glove_emb_dict,word2ind,ind2word = get_emb_vect(text,initglove_emb_dict,initglove_emb_list,initword2ind,initind2word)
    input_seq = text
    
    for i in range(len(text)):
        input_seq[i] = [word2ind[word.lower()] for word in text[i].split()]
    
    xx_tensor = [torch.LongTensor(x) for x in input_seq]    
    x_padded = pad_sequence(xx_tensor, batch_first=True, padding_value=0)    
    x_lens = [len(x) for x in text]    
    input_text = glove_emb_vec[x_padded]
    input_text = torch.tensor(input_text,dtype=torch.float).to(device)
    model.eval() # eval mode    
    out, hidden = model(input_text,x_lens)    
    prob = nn.functional.softmax(out, dim=1).data
    # Taking the class with the highest probability score from the output    
    _,word_ind = torch.max(prob, dim=1)    
    #print(word_ind)
    final_ind = []
    for line in range(len(or_text)):                
        sent = or_text[line]            
        for i in range(len(sent.split())):            
            if do_print:
                print(f"{sent.split()[i]} : {tind2word[word_ind[i].item()]}")
            final_ind.append(word_ind[i].cpu().numpy().item())
            
    return final_ind,hidden

In [20]:
word_list[3]

'The parties agree as follows :'

In [21]:
len(word_ind)

NameError: name 'word_ind' is not defined

In [25]:
word_ind,h = predict(model, [test_word_list[12]],do_print=True)

ASSIGNMENT : o
7 : o
11 : o
. : o


In [23]:
test_word_list = []
test_target_list=  []
test_word_str = ""
test_target_str = ""
with open(r"C:\Users\Ant PC\Music\dataset\FIN3.txt",'r', encoding="utf8") as myfile:    
    for i, line in enumerate(myfile):        
        if ((len(line)>0 and i>0) and line not in ['\n', '\r\n']) :            
            if (len(line)>0):
                test_word_str = test_word_str+" "+line.split()[0]
                test_target_str = test_target_str+" "+line.split()[-1]                    
        if line in ['\n', '\r\n']:            
            test_word_list.append(test_word_str[1:])
            test_target_list.append(test_target_str[1:])
            test_word_str=""
            test_target_str=""


In [26]:
len(test_word_list[2].split())

22

In [27]:
len(test_target_list[2].split())

22

In [28]:
"Stanley".lower()

'stanley'

In [29]:
test_target_list[1]

'O O O O I-ORG I-ORG I-ORG I-ORG O I-ORG I-ORG I-ORG O O O O O O I-PER I-PER O O O O I-ORG I-ORG I-ORG I-ORG O I-ORG I-ORG O O O'

In [30]:
test_target_ind = []
for sent in test_target_list[1:]:
    if (len(sent.split())>1):
        for word in sent.split():
            #print(word)
            test_target_ind.append(tword2ind[word.lower()])

In [56]:
len(test_target_ind)

13246

In [138]:
for i,ele in enumerate(test_word_list):
    print(i)
    print(ele)

0

1
This LOAN AND SECURITY AGREEMENT dated January 27 , 1999 , between SILICON VALLEY BANK (" Bank "), a California - chartered bank with its principal place of business at 3003 Tasman Drive , Santa Clara , California 95054 with a loan production office located at 40 William St ., Ste .
2
350 , Wellesley , Massachusetts 02481 doing business as " Silicon Valley East " and AKAMAI TECHNOLOGIES , INC . (" Borrower "), whose address is 201 Broadway , 4th Floor , Cambridge , Massachusetts 02139 provides the terms on which Bank will lend to Borrower and Borrower will repay Bank .
3
The parties agree as follows :
4
1 ACCOUNTING AND OTHER TERMS
5
Accounting terms not defined in this Agreement will be construed following GAAP .
6
Calculations and determinations must be made following GAAP .
7
The term " financial statements " includes the notes and schedules .
8
The terms " including " and " includes " always mean " including ( or includes ) without limitation ," in this or any Loan Document .


In [58]:
test_pred_ind=[]
for test_sent in test_word_list[1:]:
    if (len(test_sent.split())>1):
        word_ind,h = predict(model, [test_sent])
        test_pred_ind.extend(word_ind)

Subordinated : o
Loan : o
Agreement : o
- : o
Silicium : o
de : o
Provence : o
SAS : o
and : o
Evergreen : o
Solar : o
Inc : o
. : o
7 : o
- : o
December : o
2007 : o
[ : o
HERBERT : i-per
SMITH : o
LOGO : o
] : o
................................ : o
2007 : o
SILICIUM : o
DE : o
PROVENCE : o
SAS : o
and : o
EVERGREEN : o
SOLAR : o
, : o
INC : o
. : o
SUBORDINATED : o
LOAN : o
AGREEMENT : o
HERBERT : o
SMITH : o
LLP : o
Page : o
1 : o
of : o
12 : o
7 : o
- : o
December : o
2007 : o
TABLE : o
OF : o
CONTENTS : o
Clause : o
Headings : o
Page : o
1 : o
. : o
INTERPRETATION : o
3 : o
2 : o
. : o
LOAN : o
4 : o
3 : o
. : o
INTEREST : o
AND : o
REDEMPTION : o
4 : o
4 : o
. : o
EARLY : o
REDEMPTIONS : o
5 : o
5 : o
. : o
REPRESENTATIONS : o
5 : o
6 : o
. : o
UNDERTAKINGS : o
5 : o
7 : o
. : o
SUBORDINATION : o
6 : o
8 : o
. : o
COSTS : o
6 : o
9 : o
. : o
PARTIAL : o
PAYMENTS : o
6 : o
10 : o
. : o
ASSIGNMENT : o
7 : o
11 : o
. : o
WHOLE : o
AGREEMENT : o
8 : o
12 : o
. : o
VARIATION : o
AND :

2 : o
. : o
2 : o
The : o
actual : o
disbursement : o
date : o
of : o
the : o
last : o
instalment : o
of : o
the : o
Loan : o
to : o
the : o
Borrower : i-per
shall : o
be : o
the : o
" : o
Effective : o
Date : o
". : o
2 : o
. : o
3 : o
The : o
Loan : o
is : o
to : o
be : o
paid : o
by : o
the : o
Lender : i-per
into : o
the : o
Borrower : i-per
' : o
s : o
account : o
in : o
immediately : o
available : o
cleared : o
funds : o
as : o
follows : o
: : o
Bank : i-org
: : o
ING : o
Belgium : o
S : o
. : o
A : o
., : o
Succursale : o
en : o
France : o
Branch : i-org
: : o
Immeuble : o
Les : o
Caryatides : i-loc
, : o
24 : o
/ : o
26 : o
boulevard : o
Carnot : o
, : o
59042 : o
Lille : o
cedex : o
Account : o
Number : o
: : o
30438 : i-loc
IBAN : o
number : o
: : o
FR76 : o
3043 : o
8000 : i-loc
0836 : o
8300 : i-loc
3000 : o
905 : o
BIC : o
Number : o
: : o
INGBFRPP : o
Code : o
Guichet : i-org
: : o
00008 : o
No : o
du : o
Compte : o
: : o
36830 : o
03 : o
0009 : o
Cle : i-org
RIB : o
: : 

7 : o
. : o
SUBORDINATION : o
7 : o
. : o
1 : o
In : o
the : o
event : o
that : o
insolvency : o
proceedings : o
are : o
initiated : o
against : o
the : o
Borrower : i-per
or : o
that : o
it : o
is : o
unable : o
to : o
pay : o
its : o
debts : o
as : o
they : o
fall : o
due : o
or : o
in : o
the : o
event : o
of : o
any : o
moratorium : o
on : o
its : o
debts : o
or : o
if : o
the : o
Borrower : i-per
has : o
proposed : o
any : o
composition : o
, : o
assignment : o
or : o
arrangement : o
to : o
its : o
creditors : o
, : o
the : o
obligation : o
to : o
repay : o
the : o
outstanding : o
amount : o
of : o
the : o
Loan : o
: : o
7 : o
. : o
1 : o
. : o
1 : o
shall : o
be : o
subordinated : o
to : o
any : o
indebtedness : o
of : o
the : o
Borrower : i-per
to : o
any : o
lending : o
or : o
financial : o
institution : o
in : o
any : o
way : o
related : o
to : o
the : o
Works : o
, : o
both : o
present : o
and : o
future : o
notwithstanding : o
whether : o
such : o
indebtedness : o
is : o
rec

( : o
C : o
) : o
If : o
the : o
Borrower : i-per
obtains : o
the : o
Lender : i-per
' : o
s : o
prior : o
written : o
consent : o
to : o
any : o
such : o
assignment : o
, : o
transfer : o
, : o
mortgage : o
, : o
charge : o
, : o
pledge : o
or : o
other : o
encumbrance : o
under : o
sub : o
- : o
clause : o
( : o
A : o
) : o
above : o
assignment : o
the : o
Borrower : i-per
shall : o
not : o
have : o
to : o
provide : o
a : o
repayment : o
guarantee : o
to : o
Lender : i-per
and : o
the : o
Borrower : i-per
shall : o
be : o
released : o
from : o
any : o
obligation : o
hereunder : o
upon : o
the : o
execution : o
of : o
an : o
assignment : o
and : o
assumption : o
agreement : o
whereby : o
Borrower : i-per
assigns : o
all : o
of : o
its : o
rights : o
to : o
the : o
assignee : o
and : o
the : o
assignee : o
assumes : o
all : o
of : o
the : o
obligations : o
of : o
Borrower : i-per
under : o
this : o
Loan : o
Agreement : o
. : o
( : o
D : o
) : o
Successive : o
assignments : o
of : o
Bor

14 : o
. : o
2 : o
Any : o
notice : o
or : o
other : o
communication : o
shall : o
be : o
deemed : o
to : o
have : o
been : o
duly : o
given : o
: : o
( : o
A : o
) : o
if : o
delivered : o
personally : o
, : o
when : o
left : o
at : o
the : o
address : o
referred : o
to : o
in : o
Article : o
14 : o
. : o
1 : o
; : o
or : o
( : o
B : o
) : o
if : o
delivered : o
by : o
commercial : o
courier : o
, : o
on : o
the : o
date : o
of : o
signature : o
of : o
the : o
courier : o
' : o
s : o
receipt : o
; : o
or : o
( : o
C : o
) : o
if : o
sent : o
by : o
fax : o
, : o
at : o
the : o
time : o
of : o
transmission : o
; : o
or : o
( : o
D : o
) : o
if : o
sent : o
by : o
e : o
- : o
mail : o
, : o
at : o
the : o
time : o
of : o
despatch : o
; : o
( : o
E : o
) : o
if : o
deemed : o
receipt : o
under : o
the : o
previous : o
paragraphs : o
of : o
this : o
Article : o
16 : o
. : o
2 : o
is : o
not : o
within : o
business : o
hours : o
( : o
meaning : o
9 : o
. : o
00 : o
am : o
to : o
5 : o
. : 

...

Each : o
Holder : o
agrees : o
that : o
if : o
at : o
any : o
time : o
it : o
shall : o
receive : o
from : o
any : o
source : o
whatsoever : o
( : o
other : o
than : o
as : o
a : o
distribution : o
from : o
the : o
Servicer : o
to : o
which : o
it : o
is : o
entitled : o
hereunder : o
) : o
any : o
payment : o
on : o
account : o
of : o
the : o
Loan : o
in : o
excess : o
of : o
amounts : o
due : o
such : o
Holder : o
hereunder : o
, : o
it : o
will : o
promptly : o
remit : o
such : o
excess : o
to : o
the : o
Servicer : o
. : o
The : o
Servicer : o
shall : o
have : o
the : o
right : o
to : o
offset : o
any : o
amounts : o
due : o
hereunder : o
from : o
any : o
Holder : o
7 : o
with : o
respect : o
to : o
the : o
Loan : o
against : o
any : o
future : o
payments : o
due : o
to : o
such : o
Holder : o
from : o
the : o
Loan : o
; : o
provided : o
, : o
that : o
the : o
obligations : o
of : o
each : o
Holder : o
under : o
this : o
Section : o
3 : o
are : o
separate : o
and : o
distinct : o
o

( : o
c : o
) : o
Appointment : o
of : o
Special : o
Servicer : o
. : o
The : o
Holders : o
shall : o
have : o
the : o
right : o
to : o
appoint : o
a : o
special : o
servicer : o
upon : o
two : o
( : o
2 : o
) : o
Business : o
Days : o
written : o
notice : o
to : o
Servicer : o
with : o
respect : o
to : o
any : o
Investment : o
that : o
is : o
subject : o
to : o
an : o
Event : o
of : o
Default : o
. : o
In : o
the : o
event : o
that : o
a : o
special : o
servicer : o
is : o
appointed : o
with : o
respect : o
to : o
an : o
Investment : o
, : o
( : o
1 : o
) : o
Servicer : o
shall : o
cooperate : o
with : o
the : o
Holders : o
and : o
any : o
special : o
servicer : o
in : o
the : o
transition : o
of : o
such : o
servicing : o
obligations : o
, : o
including : o
, : o
without : o
limitation : o
, : o
turning : o
over : o
all : o
Loan : o
Documents : o
in : o
Servicer : o
' : o
s : o
possession : o
with : o
respect : o
to : o
such : o
Investment : o
to : o
such : o
special : o
servicer : o

EACH : o
OF : o
THE : o
PARTIES : o
HEREBY : o
IRREVOCABLY : o
WAIVES : o
ALL : o
RIGHT : o
TO : o
TRIAL : o
BY : o
JURY : o
IN : o
ANY : o
ACTION : o
, : o
PROCEEDING : o
OR : o
COUNTERCLAIM : o
ARISING : o
OUT : o
OF : o
OR : o
RELATING : o
TO : o
THIS : o
AGREEMENT : o
. : o
SECTION : o
12 : o
. : o
MODIFICATIONS : o
. : o
This : o
Agreement : o
shall : o
not : o
be : o
modified : o
, : o
cancelled : o
or : o
terminated : o
except : o
by : o
an : o
instrument : o
in : o
writing : o
signed : o
by : o
the : o
parties : o
hereto : o
. : o
The : o
party : o
seeking : o
modification : o
of : o
this : o
Agreement : o
shall : o
be : o
solely : o
responsible : o
for : o
any : o
and : o
all : o
expenses : o
that : o
may : o
arise : o
in : o
order : o
to : o
modify : o
this : o
Agreement : o
. : o
SECTION : o
13 : o
. : o
SUCCESSORS : o
AND : o
ASSIGNS : o
; : o
THIRD : o
PARTY : o
BENEFICIARIES : o
. : o
This : o
Agreement : o
shall : o
inure : o
to : o
the : o
benefit : o
of : o
and : o
be 

Loan : o
Agreement : o
[ : o
Amendment : o
No : o
. : o
1 : o
] : o
- : o
Union : o
Bank : i-org
of : o
California : i-loc
NA : o
and : o
Crocs : o
Inc : o
. : o
[ : o
UNION : o
BANK : i-org
OF : o
CALIFORNIA : i-loc
LOGO : o
] : o
AMENDMENT : o
NO : o
. : o
1 : o
TO : o
LOAN : o
AGREEMENT : o
THIS : o
AMENDMENT : o
NO : o
. : o
1 : o
TO : o
LOAN : o
AGREEMENT : o
( : o
this : o
" : o
Amendment : o
"), : o
dated : o
as : o
of : o
November : o
21 : o
, : o
2007 : o
, : o
is : o
entered : o
into : o
by : o
and : o
among : o
Union : i-org
Bank : i-org
of : o
California : i-loc
, : o
N : o
. : o
A : o
., : o
(" : o
Bank : i-org
"), : o
and : o
Crocs : o
, : o
Inc : o
., : o
a : o
Delaware : i-org
corporation : i-org
(" : o
Borrower : i-per
"), : o
with : o
reference : o
to : o
the : o
following : o
facts : o
: : o
RECITALS : o
A : o
. : o
The : o
Borrower : i-per
and : o
Bank : i-org
are : o
parties : o
to : o
that : o
certain : o
Loan : o
Agreement : o
, : o
dated : o
as : o
of : o
May : 

( : o
g : o
) : o
Successors : o
and : o
Assigns : o
. : o
This : o
Amendment : o
is : o
binding : o
upon : o
and : o
shall : o
inure : o
to : o
the : o
benefit : o
of : o
the : o
Bank : i-org
and : o
the : o
Borrower : i-per
and : o
their : o
respective : o
successors : o
and : o
assigns : o
; : o
provided : o
, : o
however : o
, : o
that : o
the : o
Borrower : i-per
may : o
not : o
assign : o
or : o
transfer : o
any : o
of : o
its : o
rights : o
or : o
obligations : o
hereunder : o
without : o
the : o
prior : o
written : o
consent : o
of : o
the : o
Bank : i-org
. : o
( : o
h : o
) : o
Counterparts : o
. : o
This : o
Amendment : o
may : o
be : o
executed : o
in : o
one : o
or : o
more : o
counterparts : o
, : o
each : o
of : o
which : o
when : o
so : o
executed : o
shall : o
be : o
deemed : o
to : o
be : o
an : o
original : o
, : o
but : o
all : o
of : o
which : o
when : o
taken : o
together : o
shall : o
constitute : o
one : o
and : o
the : o
same : o
instrument : o
. : o
( : o
i : 

In [105]:
tind2word

{0: '<pad>', 1: 'o', 2: 'i-misc', 3: 'i-loc', 4: 'i-per', 5: 'i-org'}

In [ ]:
["Stanley went to India 25th Aug","my country is Chennai","some random text","now is next batch"]

In [128]:
word_ind

[3, 1, 1, 2, 2, 4, 3, 1, 1, 2, 3, 1, 1, 3, 1, 1, 2]

In [129]:
truth = np.array([3,1,1,2,4,4,1,2,1,2,1,1,1,1,1,1,1])
pred = np.array(word_ind)

In [135]:
truth

array([3, 1, 1, 2, 4, 4, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1])

In [104]:
word_ind

tensor([3, 1, 1, 2, 2, 4, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2],
       device='cuda:0')

In [175]:
len(test_target_ind)

40997

In [59]:
from sklearn.metrics import f1_score,confusion_matrix,classification_report
f1_score(test_target_ind, test_pred_ind, average='macro')

0.5611938677660733

In [60]:
f1_score(test_target_ind, test_pred_ind, average='micro')

0.9824097840857617

In [65]:
#report = classification_report(y_test, y_pred, output_dict=True)
report = classification_report(test_target_ind, test_pred_ind)

In [66]:
print(report)

              precision    recall  f1-score   support

           1       0.99      0.86      0.92       236
           2       0.72      0.49      0.59        79
           3       0.00      0.00      0.00         7
           4       0.99      0.99      0.99     12810
           5       0.35      0.28      0.31       114

    accuracy                           0.98     13246
   macro avg       0.61      0.52      0.56     13246
weighted avg       0.98      0.98      0.98     13246



In [67]:
confusion_matrix(test_target_ind, test_pred_ind)

array([[  202,     1,     0,    33,     0],
       [    1,    39,     0,    34,     5],
       [    0,     0,     0,     7,     0],
       [    2,    14,     0, 12740,    54],
       [    0,     0,     0,    82,    32]], dtype=int64)

In [68]:
tind2word

{0: '<pad>', 1: 'i-per', 2: 'i-loc', 3: 'i-misc', 4: 'o', 5: 'i-org'}

In [159]:
set(test_target_ind)

{1, 2, 3, 4, 5}

In [160]:
set(test_pred_ind)

{1, 2, 3, 4, 5}

In [157]:
tind2word

{0: '<pad>', 1: 'o', 2: 'i-misc', 3: 'i-loc', 4: 'i-per', 5: 'i-org'}

In [170]:
train_pred_ind=[]
for test_sent in word_list[1:]:
    if (len(test_sent.split())>1):
        word_ind,h = predict(model, [test_sent])
        test_pred_ind.extend(word_ind)

Subordinated : o
Loan : o
Agreement : o
- : o
Silicium : i-org
de : i-org
Provence : i-org
SAS : i-org
and : o
Evergreen : i-org
Solar : i-org
Inc : i-org
. : o
7 : o
- : o
December : o
2007 : o
[ : o
HERBERT : i-per
SMITH : i-per
LOGO : o
] : o
................................ : o
2007 : o
SILICIUM : i-org
DE : i-org
PROVENCE : i-org
SAS : i-org
and : o
EVERGREEN : i-org
SOLAR : i-org
, : o
INC : o
. : o
SUBORDINATED : o
LOAN : o
AGREEMENT : o
HERBERT : i-per
SMITH : i-per
LLP : o
Page : o
1 : o
of : o
12 : o
7 : o
- : o
December : o
2007 : o
TABLE : o
OF : o
CONTENTS : o
Clause : o
Headings : o
Page : o
1 : o
. : o
INTERPRETATION : o
3 : o
2 : o
. : o
LOAN : o
4 : o
3 : o
. : o
INTEREST : o
AND : o
REDEMPTION : o
4 : o
4 : o
. : o
EARLY : o
REDEMPTIONS : o
5 : o
5 : o
. : o
REPRESENTATIONS : o
5 : o
6 : o
. : o
UNDERTAKINGS : o
5 : o
7 : o
. : o
SUBORDINATION : o
6 : o
8 : o
. : o
COSTS : o
6 : o
9 : o
. : o
PARTIAL : o
PAYMENTS : o
6 : o
10 : o
. : o
ASSIGNMENT : o
7 : o
11 : o
. : 

2 : o
. : o
2 : o
The : o
actual : o
disbursement : o
date : o
of : o
the : o
last : o
instalment : o
of : o
the : o
Loan : o
to : o
the : o
Borrower : i-per
shall : o
be : o
the : o
" : o
Effective : o
Date : o
". : o
2 : o
. : o
3 : o
The : o
Loan : o
is : o
to : o
be : o
paid : o
by : o
the : o
Lender : i-per
into : o
the : o
Borrower : i-per
' : o
s : o
account : o
in : o
immediately : o
available : o
cleared : o
funds : o
as : o
follows : o
: : o
Bank : i-org
: : o
ING : o
Belgium : i-loc
S : o
. : o
A : o
., : o
Succursale : o
en : o
France : i-loc
Branch : o
: : o
Immeuble : i-loc
Les : i-loc
Caryatides : i-loc
, : o
24 : i-loc
/ : i-loc
26 : i-loc
boulevard : i-loc
Carnot : i-loc
, : o
59042 : i-loc
Lille : i-loc
cedex : i-loc
Account : o
Number : o
: : o
30438 : o
IBAN : o
number : o
: : o
FR76 : o
3043 : o
8000 : o
0836 : o
8300 : o
3000 : o
905 : o
BIC : o
Number : o
: : o
INGBFRPP : o
Code : o
Guichet : o
: : o
00008 : o
No : o
du : o
Compte : o
: : o
36830 : o
03 : o
0009 

7 : o
. : o
SUBORDINATION : o
7 : o
. : o
1 : o
In : o
the : o
event : o
that : o
insolvency : o
proceedings : o
are : o
initiated : o
against : o
the : o
Borrower : i-per
or : o
that : o
it : o
is : o
unable : o
to : o
pay : o
its : o
debts : o
as : o
they : o
fall : o
due : o
or : o
in : o
the : o
event : o
of : o
any : o
moratorium : o
on : o
its : o
debts : o
or : o
if : o
the : o
Borrower : i-per
has : o
proposed : o
any : o
composition : o
, : o
assignment : o
or : o
arrangement : o
to : o
its : o
creditors : o
, : o
the : o
obligation : o
to : o
repay : o
the : o
outstanding : o
amount : o
of : o
the : o
Loan : o
: : o
7 : o
. : o
1 : o
. : o
1 : o
shall : o
be : o
subordinated : o
to : o
any : o
indebtedness : o
of : o
the : o
Borrower : i-per
to : o
any : o
lending : o
or : o
financial : o
institution : o
in : o
any : o
way : o
related : o
to : o
the : o
Works : o
, : o
both : o
present : o
and : o
future : o
notwithstanding : o
whether : o
such : o
indebtedness : o
is : o
rec

( : o
C : o
) : o
If : o
the : o
Borrower : i-per
obtains : o
the : o
Lender : i-per
' : o
s : o
prior : o
written : o
consent : o
to : o
any : o
such : o
assignment : o
, : o
transfer : o
, : o
mortgage : o
, : o
charge : o
, : o
pledge : o
or : o
other : o
encumbrance : o
under : o
sub : o
- : o
clause : o
( : o
A : o
) : o
above : o
assignment : o
the : o
Borrower : i-per
shall : o
not : o
have : o
to : o
provide : o
a : o
repayment : o
guarantee : o
to : o
Lender : i-per
and : o
the : o
Borrower : i-per
shall : o
be : o
released : o
from : o
any : o
obligation : o
hereunder : o
upon : o
the : o
execution : o
of : o
an : o
assignment : o
and : o
assumption : o
agreement : o
whereby : o
Borrower : i-per
assigns : o
all : o
of : o
its : o
rights : o
to : o
the : o
assignee : o
and : o
the : o
assignee : o
assumes : o
all : o
of : o
the : o
obligations : o
of : o
Borrower : i-per
under : o
this : o
Loan : o
Agreement : o
. : o
( : o
D : o
) : o
Successive : o
assignments : o
of : o
Bor

14 : o
. : o
2 : o
Any : o
notice : o
or : o
other : o
communication : o
shall : o
be : o
deemed : o
to : o
have : o
been : o
duly : o
given : o
: : o
( : o
A : o
) : o
if : o
delivered : o
personally : o
, : o
when : o
left : o
at : o
the : o
address : o
referred : o
to : o
in : o
Article : o
14 : o
. : o
1 : o
; : o
or : o
( : o
B : o
) : o
if : o
delivered : o
by : o
commercial : o
courier : o
, : o
on : o
the : o
date : o
of : o
signature : o
of : o
the : o
courier : o
' : o
s : o
receipt : o
; : o
or : o
( : o
C : o
) : o
if : o
sent : o
by : o
fax : o
, : o
at : o
the : o
time : o
of : o
transmission : o
; : o
or : o
( : o
D : o
) : o
if : o
sent : o
by : o
e : o
- : o
mail : o
, : o
at : o
the : o
time : o
of : o
despatch : o
; : o
( : o
E : o
) : o
if : o
deemed : o
receipt : o
under : o
the : o
previous : o
paragraphs : o
of : o
this : o
Article : o
16 : o
. : o
2 : o
is : o
not : o
within : o
business : o
hours : o
( : o
meaning : o
9 : o
. : o
00 : o
am : o
to : o
5 : o
. : 

...

Each : o
Holder : o
agrees : o
that : o
if : o
at : o
any : o
time : o
it : o
shall : o
receive : o
from : o
any : o
source : o
whatsoever : o
( : o
other : o
than : o
as : o
a : o
distribution : o
from : o
the : o
Servicer : o
to : o
which : o
it : o
is : o
entitled : o
hereunder : o
) : o
any : o
payment : o
on : o
account : o
of : o
the : o
Loan : o
in : o
excess : o
of : o
amounts : o
due : o
such : o
Holder : o
hereunder : o
, : o
it : o
will : o
promptly : o
remit : o
such : o
excess : o
to : o
the : o
Servicer : o
. : o
The : o
Servicer : o
shall : o
have : o
the : o
right : o
to : o
offset : o
any : o
amounts : o
due : o
hereunder : o
from : o
any : o
Holder : o
7 : o
with : o
respect : o
to : o
the : o
Loan : o
against : o
any : o
future : o
payments : o
due : o
to : o
such : o
Holder : o
from : o
the : o
Loan : o
; : o
provided : o
, : o
that : o
the : o
obligations : o
of : o
each : o
Holder : o
under : o
this : o
Section : o
3 : o
are : o
separate : o
and : o
distinct : o
o

( : o
c : o
) : o
Appointment : o
of : o
Special : o
Servicer : o
. : o
The : o
Holders : o
shall : o
have : o
the : o
right : o
to : o
appoint : o
a : o
special : o
servicer : o
upon : o
two : o
( : o
2 : o
) : o
Business : o
Days : o
written : o
notice : o
to : o
Servicer : o
with : o
respect : o
to : o
any : o
Investment : o
that : o
is : o
subject : o
to : o
an : o
Event : o
of : o
Default : o
. : o
In : o
the : o
event : o
that : o
a : o
special : o
servicer : o
is : o
appointed : o
with : o
respect : o
to : o
an : o
Investment : o
, : o
( : o
1 : o
) : o
Servicer : o
shall : o
cooperate : o
with : o
the : o
Holders : o
and : o
any : o
special : o
servicer : o
in : o
the : o
transition : o
of : o
such : o
servicing : o
obligations : o
, : o
including : o
, : o
without : o
limitation : o
, : o
turning : o
over : o
all : o
Loan : o
Documents : o
in : o
Servicer : o
' : o
s : o
possession : o
with : o
respect : o
to : o
such : o
Investment : o
to : o
such : o
special : o
servicer : o

EACH : o
OF : o
THE : o
PARTIES : o
HEREBY : o
IRREVOCABLY : o
WAIVES : o
ALL : o
RIGHT : o
TO : o
TRIAL : o
BY : o
JURY : o
IN : o
ANY : o
ACTION : o
, : o
PROCEEDING : o
OR : o
COUNTERCLAIM : o
ARISING : o
OUT : o
OF : o
OR : o
RELATING : o
TO : o
THIS : o
AGREEMENT : o
. : o
SECTION : o
12 : o
. : o
MODIFICATIONS : o
. : o
This : o
Agreement : o
shall : o
not : o
be : o
modified : o
, : o
cancelled : o
or : o
terminated : o
except : o
by : o
an : o
instrument : o
in : o
writing : o
signed : o
by : o
the : o
parties : o
hereto : o
. : o
The : o
party : o
seeking : o
modification : o
of : o
this : o
Agreement : o
shall : o
be : o
solely : o
responsible : o
for : o
any : o
and : o
all : o
expenses : o
that : o
may : o
arise : o
in : o
order : o
to : o
modify : o
this : o
Agreement : o
. : o
SECTION : o
13 : o
. : o
SUCCESSORS : o
AND : o
ASSIGNS : o
; : o
THIRD : o
PARTY : o
BENEFICIARIES : o
. : o
This : o
Agreement : o
shall : o
inure : o
to : o
the : o
benefit : o
of : o
and : o
be 

Loan : o
Agreement : o
[ : o
Amendment : o
No : o
. : o
1 : o
] : o
- : o
Union : i-org
Bank : i-org
of : o
California : i-loc
NA : o
and : o
Crocs : o
Inc : o
. : o
[ : o
UNION : i-org
BANK : i-org
OF : o
CALIFORNIA : i-loc
LOGO : o
] : o
AMENDMENT : o
NO : o
. : o
1 : o
TO : o
LOAN : o
AGREEMENT : o
THIS : o
AMENDMENT : o
NO : o
. : o
1 : o
TO : o
LOAN : o
AGREEMENT : o
( : o
this : o
" : o
Amendment : o
"), : o
dated : o
as : o
of : o
November : o
21 : o
, : o
2007 : o
, : o
is : o
entered : o
into : o
by : o
and : o
among : o
Union : i-org
Bank : i-org
of : o
California : i-loc
, : o
N : o
. : o
A : o
., : o
(" : o
Bank : i-org
"), : o
and : o
Crocs : o
, : o
Inc : o
., : o
a : o
Delaware : i-org
corporation : o
(" : o
Borrower : i-per
"), : o
with : o
reference : o
to : o
the : o
following : o
facts : o
: : o
RECITALS : o
A : o
. : o
The : o
Borrower : i-per
and : o
Bank : i-org
are : o
parties : o
to : o
that : o
certain : o
Loan : o
Agreement : o
, : o
dated : o
as : o
of : o
Ma

( : o
g : o
) : o
Successors : o
and : o
Assigns : o
. : o
This : o
Amendment : o
is : o
binding : o
upon : o
and : o
shall : o
inure : o
to : o
the : o
benefit : o
of : o
the : o
Bank : i-org
and : o
the : o
Borrower : i-per
and : o
their : o
respective : o
successors : o
and : o
assigns : o
; : o
provided : o
, : o
however : o
, : o
that : o
the : o
Borrower : i-per
may : o
not : o
assign : o
or : o
transfer : o
any : o
of : o
its : o
rights : o
or : o
obligations : o
hereunder : o
without : o
the : o
prior : o
written : o
consent : o
of : o
the : o
Bank : i-org
. : o
( : o
h : o
) : o
Counterparts : o
. : o
This : o
Amendment : o
may : o
be : o
executed : o
in : o
one : o
or : o
more : o
counterparts : o
, : o
each : o
of : o
which : o
when : o
so : o
executed : o
shall : o
be : o
deemed : o
to : o
be : o
an : o
original : o
, : o
but : o
all : o
of : o
which : o
when : o
taken : o
together : o
shall : o
constitute : o
one : o
and : o
the : o
same : o
instrument : o
. : o
( : o
i : 

In [185]:
len(test_pred_ind[41116:])

13246

In [178]:
54362 - 13246

41116

In [173]:
train_target_ind = []
for sent in target_list[1:]:
    if (len(sent.split())>1):
        for word in sent.split():
            #print(word)
            train_target_ind.append(tword2ind[word.lower()])

In [174]:
len(train_target_ind)

13246

In [186]:
test_target_ind[41116:]

[]

In [188]:
f1_score(train_target_ind, test_pred_ind[41116:], average='macro')

1.0

In [190]:
print(classification_report(train_target_ind, test_pred_ind[41116:]))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     12810
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00        79
           4       1.00      1.00      1.00       236
           5       1.00      1.00      1.00       114

    accuracy                           1.00     13246
   macro avg       1.00      1.00      1.00     13246
weighted avg       1.00      1.00      1.00     13246



In [ ]:
from pprint import pprint as pp

In [58]:
tt = ["Stanley went to India 25th Aug","ooty"]
input_seq = tt
print(type(tt))
for line in range(len(tt)):
        sent = tt[line]
        print(sent)

<class 'list'>
Stanley went to India 25th Aug
ooty


In [42]:
input_seq

[[3634, 389, 5, 475, 8962, 22552]]

In [ ]:
tint2char

In [ ]:

def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [ ]:
sample(model, 16, 'how')

In [ ]:
sample(model, 16, 'how')

In [ ]:
original = [[31, 48],
   [6, 0, 52, 0, 12, 2, 7],
   [42, 0, 6, 14, 1, 3],
   [12, 1, 9, 9],
   [2, 5],
   [21, 5, 19, 12, 5, 2, 2],
   [36, 6, 4, 2, 4, 7, 11],
   [9, 1, 14, 21],
   [18]]

In [44]:
data = {'chars': [[[31, 48],
   [6, 0, 52, 0, 12, 2, 7],
   [42, 0, 6, 14, 1, 3],
   [12, 1, 9, 9],
   [2, 5],
   [21, 5, 19, 12, 5, 2, 2],
   [36, 6, 4, 2, 4, 7, 11],
   [9, 1, 14, 21],
   [18]],
                 [[32, 40],
   [16, 0, 50, 0, 1, 20, 17],
   [40, 0, 16, 14, 11, 13],
   [12, 1, 19, 9],
   [2, 5],
   [21, 5, 19, 12, 5, 2, 2],
   [36, 6, 4, 2, 4, 7, 11],
   [9, 1, 14, 21],
   [18]]],
   'words': [944, 15473, 198, 590, 8, 3848, 207, 6233, 2]}

In [45]:
sentence_in = data['words']

In [46]:
chars3 = data['chars']

In [57]:
for chars2 in chars3:
    print(chars2)

[[31, 48], [6, 0, 52, 0, 12, 2, 7], [42, 0, 6, 14, 1, 3], [12, 1, 9, 9], [2, 5], [21, 5, 19, 12, 5, 2, 2], [36, 6, 4, 2, 4, 7, 11], [9, 1, 14, 21], [18]]
[[32, 40], [16, 0, 50, 0, 1, 20, 17], [40, 0, 16, 14, 11, 13], [12, 1, 19, 9], [2, 5], [21, 5, 19, 12, 5, 2, 2], [36, 6, 4, 2, 4, 7, 11], [9, 1, 14, 21], [18]]


In [58]:
d = {}
l= []

for chars2 in chars3:
    ## Padding the each word to max word size of that sentence
    chars2_length = [len(c) for c in chars2]
    char_maxl = max(chars2_length)
    chars2_mask = np.zeros((len(chars2_length), char_maxl), dtype='int')
    for i, c in enumerate(chars2):
        chars2_mask[i, :chars2_length[i]] = c
    l.append(chars2_mask)

In [59]:
l

[array([[31, 48,  0,  0,  0,  0,  0],
        [ 6,  0, 52,  0, 12,  2,  7],
        [42,  0,  6, 14,  1,  3,  0],
        [12,  1,  9,  9,  0,  0,  0],
        [ 2,  5,  0,  0,  0,  0,  0],
        [21,  5, 19, 12,  5,  2,  2],
        [36,  6,  4,  2,  4,  7, 11],
        [ 9,  1, 14, 21,  0,  0,  0],
        [18,  0,  0,  0,  0,  0,  0]]),
 array([[32, 40,  0,  0,  0,  0,  0],
        [16,  0, 50,  0,  1, 20, 17],
        [40,  0, 16, 14, 11, 13,  0],
        [12,  1, 19,  9,  0,  0,  0],
        [ 2,  5,  0,  0,  0,  0,  0],
        [21,  5, 19, 12,  5,  2,  2],
        [36,  6,  4,  2,  4,  7, 11],
        [ 9,  1, 14, 21,  0,  0,  0],
        [18,  0,  0,  0,  0,  0,  0]])]

In [36]:
len(l)

18

In [90]:
chars2_mask = torch.tensor(l[0])

In [91]:
sentence_in

[944, 15473, 198, 590, 8, 3848, 207, 6233, 2]

In [92]:
char_embeds = nn.Embedding(56, 10)

In [93]:
char_embeds

Embedding(56, 10)

In [94]:
chars_embeds_new = char_embeds(chars2_mask).unsqueeze(1)

In [95]:
chars_embeds_new.shape

torch.Size([9, 1, 7, 10])

In [96]:
char_cnn3 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 10), padding=(2,0))

In [97]:
char_cnn3

Conv2d(1, 10, kernel_size=(3, 10), stride=(1, 1), padding=(2, 0))

In [98]:
chars_cnn_out3 = char_cnn3(chars_embeds_new)

In [99]:
chars_cnn_out3.shape

torch.Size([9, 10, 9, 1])